# Importing libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
import regex as re


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Context

dataset:  https://www.kaggle.com/zeyadkhalid/mbti-personality-types-500-dataset

https://www.kaggle.com/code/mercurio117/mbti-500/data

https://www.kaggle.com/datasets/datasnaek/mbti-type

# Content

    ~106K records of preprocessed posts and their authors' personality types.
    
    Posts are equal-sized: 500 words per sample   

# About the dataset

Posts are preprocessed texts:

    - No punctuations, stopwords, URLs
    
    - Lemmatization
    
    - Reconstruct samples to be equal-sized chunks (500 words per sample)
    
Personality types are 16 unique values

# Read the dataset into a pandas dataframe

Now it's time to read the dataset and make a simple exploratory analysis

In [31]:
# train 하게 될 데이터 셋 이게 첫번째 데이터셋
df = pd.read_csv('drive/MyDrive/Colab Notebooks/MBTI 500.csv')

# validation 하게 될 데이터 셋 두번째 데이터셋 + 전처리
# dftest = pd.read_csv('drive/MyDrive/Colab Notebooks/mbti_1.csv')

# 첫번째 시도
# dftest = pd.read_csv('drive/MyDrive/Colab Notebooks/our_test.csv')

# 네이버를 포함한 두번째 시도
# dftest = pd.read_csv('drive/MyDrive/Colab Notebooks/add_naver.csv')

# 슬랙 데이터
dftest = pd.read_csv('drive/MyDrive/Colab Notebooks/our_slack.csv')

# dftest = pd.read_csv('drive/MyDrive/Colab Notebooks/team3 SLACK.xlsx')

In [32]:
for i in range(len(dftest["mbti"])):
  if not (dftest["mbti"][i] == "enfj" or dftest["mbti"][i] == "enfp" or dftest["mbti"][i] == "infj" or dftest["mbti"][i] == "isfp"):
    dftest["mbti"][i] = "entj"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
# 데이터 확인 
# df.head()
dftest.head()

,Unnamed: 0,text,EN,NAVER,mbti
0,0,안녕하세요! 앞으로 함께 프로젝트를 진행하게 될 3팀입니다! 반가워요!,Good day to you too! We are the three teams th...,Hello! We are the 3 teams who will work on the...,enfj
1,1,안지오님이랑 빙창선님은 개인 사정으로 수업을 그만 들으신다고 합니다! 그리고 오늘 ...,I heard that the director and the Hwajangseon ...,Anjio and Bingchangsun are going to stop takin...,enfj
2,2,"아 그리고 프로젝트 1,2 는 팀별로 구분된다고 합니다! 링크보내드릴게요!","Oh, and the 1/2 project is called a team! I'll...",And project 1 and 2 are divided into teams! I'...,enfj
3,3,지금 라이브 강의 하고 있나요…?,Are you currently on a live lecture?,Are you giving a live lecture now...?,enfj
4,4,여러분! 오늘까지는 팀장을 정해서 프로젝트 페이지 안에 있는 템플릿이나 기록을 진행...,"You guys are the best! Until today, I'll have ...","Ladies and Gentlemen! By today, we need to sel...",enfj


In [5]:
# 포스팅의 내용
# df['posts'][0]

'know intj tool use interaction people excuse antisocial truly enlighten mastermind know would count pet peeze something time matter people either whether group people mall never see best friend sit outside conversation jsut listen want interject sit formulate say wait inject argument thought find fascinate sit watch people talk people fascinate sit class watch different people find intrigue dad intj u stand look like line safeway watch people home talk people like think military job people voluntarily go job important show deference endanger live glorify way civilian think pretty ignorant general think military necessary defense mechanism political tactic feel like u specifically invest much money could put money education whatnot though personally sound budget aernative really comment one way base two politician eye year ago come name somewhat important kinda role model nowadays pick keep score individual level mean little vary accord number condition day may score high others low sw

In [6]:
# mbti 
# df['type'][0]

'INTJ'

In [ ]:
# df['type'].unique()

array(['INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP', 'ENFJ', 'ENFP',
       'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP'],
      dtype=object)

In [ ]:
# print(f"Total of {len(df['type'].unique())} types of classified MBTI posts")

Total of 16 types of classified MBTI posts


# Checking null values

In [ ]:
# df.isnull().sum()

posts    0
type     0
dtype: int64

# Checking the number of posts per type

In [34]:
# Mbti 분포
df_bar_chart=dftest.groupby('mbti').count()


trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['NAVER'])

data = [trace1]
layout = go.Layout(title='MBTI # Classified Posts per Type')

fig = go.Figure(data=data, layout=layout)

fig.show()

# Recreate the model?

This machine learning model takes it's time to train data

To avoid waiting every time, We're going to use the feature dump/load from pickle

In [35]:
# Flag to re-create or not the machine learning model
recreate_model=False

In [36]:
# We'll save the model into a file:
filename = 'mbti_svm_v2.sav'

In [37]:
# If the model file doesn't exists
if not os.path.isfile(filename):
    recreate_model=True

# Model

The machine learning supervised model that we'll use here is a Classification kind, named Support Vector Machine

References:

 - https://en.wikipedia.org/wiki/Support-vector_machine

- https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

### Model Pipeline

We're going to need a pipelined model ir order to facilitate the entire process of CountVectorizer (TfidfVectorizer) and svm.LinearSVC model

To do that, we're going to use the Pipeline feature from sklearn.pipeline

References: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

# Split the data into train and test

In [38]:
# 데이터셋 중에서 심볼이나 url 등을 지운다
def clean_text(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    cleaned_text=[]
    # for sentence in tqdm(data.posts):
    for sentence in tqdm(data.NAVER):
        sentence=sentence.lower()
        
        # url 등을 지운다
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
        # 글자가 아닌 심볼들을 지운다
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text

In [39]:
# 데이터 전처리
dftest.posts = clean_text(dftest)
dftest

  0%|          | 0/339 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



,Unnamed: 0,text,EN,NAVER,mbti
0,0,안녕하세요! 앞으로 함께 프로젝트를 진행하게 될 3팀입니다! 반가워요!,Good day to you too! We are the three teams th...,Hello! We are the 3 teams who will work on the...,enfj
1,1,안지오님이랑 빙창선님은 개인 사정으로 수업을 그만 들으신다고 합니다! 그리고 오늘 ...,I heard that the director and the Hwajangseon ...,Anjio and Bingchangsun are going to stop takin...,enfj
2,2,"아 그리고 프로젝트 1,2 는 팀별로 구분된다고 합니다! 링크보내드릴게요!","Oh, and the 1/2 project is called a team! I'll...",And project 1 and 2 are divided into teams! I'...,enfj
3,3,지금 라이브 강의 하고 있나요…?,Are you currently on a live lecture?,Are you giving a live lecture now...?,enfj
4,4,여러분! 오늘까지는 팀장을 정해서 프로젝트 페이지 안에 있는 템플릿이나 기록을 진행...,"You guys are the best! Until today, I'll have ...","Ladies and Gentlemen! By today, we need to sel...",enfj
...,...,...,...,...,...
334,334,6 번 공통의견에 ~~같앗습니다 말고 있었습니다! 어떻습니까,I think it's good for a common sense six times...,No.6. It's the same as the common opinion I di...,entj
335,335,"사다리뽑기로 이런 팀장님 뽑다니, 앞으로 결정사항을 사다리뽑기많이해야겟어요",I think we should pull this off by collecting ...,I can't believe I'm picking a team leader like...,entj
336,336,저는 발표 이후에 다시 찾아뵐게요!,I'll come back after the presentation!,I'll come back after the presentation!,entj
337,337,화이팅화이팅!,We are having a great day!,"Fighting, fighting!",entj


In [ ]:
# 데이터셋 하나로 train, validation 할 경우
# X = df['posts'] # features
# y = df['type']  # labels
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
# train set & test set

X_train = df['posts'] # features
y_train = df['type']  # labels

X_test = dftest['NAVER']
y_test = dftest['mbti']

# Training the model, save it to disk and open to make predictions

In [41]:
# Check if need to recreate the model
if recreate_model:    
    
    # Creating an instance to vectorizer:
    vectorizer = TfidfVectorizer()
    
    # Training the vectorizer:
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Training the classifier:
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # Pipelining the vectorizer and the classifier
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # saving the model to disk
    pickle.dump(text_clf, open(filename, 'wb'))

# If there is no need to recreate the model, just open the file from the disk    
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

# Using the test data to make predictions and analyze the accurace of the model

In [42]:
predictions = text_clf.predict(X_test)

In [49]:
for i in range(len(predictions)):
  y_test[i] = y_test[i].upper()
  print(predictions[i],y_test[i])

INTJ ENFJ
ESTP ENFJ
ENTP ENFJ
INFP ENFJ
INTJ ENFJ
INTJ ENFJ
ENFP ENFJ
INTP ENFJ
ENTP ENFJ
INTP ENFJ
INTJ ENFJ
INTP ENFJ
ESTP ENFJ
ISTJ ENFJ
INFP ENFJ
ESTP ENFJ
INFP ENFJ
INTJ ENFJ
INFP ENFJ
ESTP ENFJ
INTJ ENFJ
INTJ ENFJ
ESTP ENFJ
ISTJ ENFJ
ISTJ ENFJ
INTP ENFJ
INFP ENFJ
INTJ ENFJ
INTP ENFJ
ESTP ENFJ
INTP ENFJ
ENTP ENFJ
ISTP ENFJ
INTJ ENFJ
INFJ ENFJ
ESTP ENFJ
ESTP ENFJ
INFP ENFJ
ENFP ENFJ
INTP ENFJ
INTP ENFJ
INTJ INFJ
ESTP INFJ
ENFP INFJ
INTP INFJ
ENFP INFJ
INTP INFJ
INTP INFJ
ENFP INFJ
ESTP INFJ
INTP INFJ
INFP INFJ
INFP INFJ
INTP INFJ
INTP INFJ
ESTP INFJ
INFP INFJ
ESTP INFJ
ESTP INFJ
ENFP INFJ
INTP INFJ
ESTP INFJ
ESTP INFJ
INFP INFJ
INTJ INFJ
INFJ INFJ
INTP INFJ
INTP INFJ
ENTJ INFJ
INTP INFJ
INFP ISFP
INTJ ISFP
ENFP ISFP
ENTP ISFP
INFP ISFP
ESTP ISFP
INTJ ISFP
INTJ ISFP
INTP ISFP
ENFP ISFP
INTP ISFP
INTJ ISFP
ESTP ISFP
ENFP ISFP
ESTP ISFP
INFJ ISFP
ESTP ISFP
INTJ ISFP
INFP ISFP
INFP ISFP
INFP ISFP
ISTP ISFP
ISTP ISFP
ESTP ISFP
INTP ISFP
INFP ISFP
INTP ISFP
ESTP ISFP
ENFP ISFP
INTP ISFP


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
rec = []
for i in range(len(predictions)):
  p = predictions[i]
  t = y_test[i]
  temp = 0
  for j in range(4):
    if p[j]==t[j]:
      temp+=1
  rec.append(temp/4)

sum(rec)/len(rec)

0.4963126843657817

In [51]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        41
        ENFP       0.26      0.08      0.12        78
        ENTJ       0.25      0.03      0.05        72
        ENTP       0.00      0.00      0.00         0
        ESTP       0.00      0.00      0.00         0
        INFJ       0.08      0.03      0.05        29
        INFP       0.00      0.00      0.00         0
        INTJ       0.00      0.00      0.00         0
        INTP       0.00      0.00      0.00         0
        ISFP       0.00      0.00      0.00       119
        ISTJ       0.00      0.00      0.00         0
        ISTP       0.00      0.00      0.00         0

    accuracy                           0.03       339
   macro avg       0.05      0.01      0.02       339
weighted avg       0.12      0.03      0.04       339



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control 

In [52]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.03



# End